<a href="https://colab.research.google.com/github/rianachatterjee04/GenAssist/blob/main/Phoenix_Neighborhood_Walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies

In [ ]:
!pip install ultralytics

!pip install yt-dlp

!pip install opencv-python

Setup and Video Download

In [ ]:
import os
import glob

# Create videos directory
os.makedirs("videos", exist_ok=True)

# Download video
VIDEO_URL = "https://www.youtube.com/watch?v=bZnjrRuBT_k"
!yt-dlp -o "videos/Phoenix_Walk.%(ext)s" {VIDEO_URL}  # Note the capital 'W'

# Find the downloaded video file - match the exact case
video_path = glob.glob("videos/Phoenix_Walk.*")[0]  # Changed to match case
print(f"Video downloaded to: {video_path}")

# Get absolute path
abs_path = os.path.abspath(video_path)
print(f"Absolute path: {abs_path}")

# Verify file exists
if os.path.exists(video_path):
    print("✅ Video file exists!")
    print(f"File size: {os.path.getsize(video_path) / (1024*1024):.2f} MB")
else:
    print("❌ Video file NOT found. Check the file name and location.")

[youtube] Extracting URL: https://www.youtube.com/watch?v=bZnjrRuBT_k
[youtube] bZnjrRuBT_k: Downloading webpage
[youtube] bZnjrRuBT_k: Downloading tv client config
[youtube] bZnjrRuBT_k: Downloading player 9c6dfc4a
[youtube] bZnjrRuBT_k: Downloading tv player API JSON
[youtube] bZnjrRuBT_k: Downloading ios player API JSON
[youtube] bZnjrRuBT_k: Downloading m3u8 information
[info] bZnjrRuBT_k: Downloading 1 format(s): 315+328
[download] Destination: videos/Phoenix_Walk.f315.webm
[download] 100% of    1.76GiB in 00:01:39 at 18.01MiB/s
[download] Destination: videos/Phoenix_Walk.f328.m4a
[download] 100% of   28.67MiB in 00:00:01 at 19.33MiB/s
[Merger] Merging formats into "videos/Phoenix_Walk.mkv"
Deleting original file videos/Phoenix_Walk.f328.m4a (pass -k to keep)
Deleting original file videos/Phoenix_Walk.f315.webm (pass -k to keep)
Video downloaded to: videos/Phoenix_Walk.mkv
Absolute path: /content/videos/Phoenix_Walk.mkv
✅ Video file exists!
File size: 1826.83 MB


Extracting frames for analysis

In [ ]:
import cv2
import os
import glob

# Find the downloaded video file
video_path = glob.glob("videos/Phoenix_Walk.*")[0]  # Use the same path from first script
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Open video file
vidcap = cv2.VideoCapture(video_path)
if not vidcap.isOpened():
    print("❌ Error: Could not open video.")
    exit()

# Get video properties
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps
print(f"Video FPS: {fps}")
print(f"Total Frames: {total_frames}")
print(f"Duration: {duration:.2f} seconds")

# Extract frames from 83s to 133s
start_sec, end_sec = 86, 133
start_frame = start_sec * fps
end_frame = min(end_sec * fps, total_frames)

vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
frame_rate = 30
count = 0

while vidcap.isOpened():
    frame_id = int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
    if frame_id > end_frame:
        break

    success, image = vidcap.read()
    if not success:
        break

    if frame_id % frame_rate == 0:
        frame_name = os.path.join(output_dir, f"frame_{frame_id}.jpg")
        cv2.imwrite(frame_name, image)
        print(f"✅ Saved: {frame_name}")
        count += 1

vidcap.release()
print(f"✅ Extracted {count} frames")

Video FPS: 60
Total Frames: 37563
Duration: 626.05 seconds
✅ Saved: frames/frame_5160.jpg
✅ Saved: frames/frame_5190.jpg
✅ Saved: frames/frame_5220.jpg
✅ Saved: frames/frame_5250.jpg
✅ Saved: frames/frame_5280.jpg
✅ Saved: frames/frame_5310.jpg
✅ Saved: frames/frame_5340.jpg
✅ Saved: frames/frame_5370.jpg
✅ Saved: frames/frame_5400.jpg
✅ Saved: frames/frame_5430.jpg
✅ Saved: frames/frame_5460.jpg
✅ Saved: frames/frame_5490.jpg
✅ Saved: frames/frame_5520.jpg
✅ Saved: frames/frame_5550.jpg
✅ Saved: frames/frame_5580.jpg
✅ Saved: frames/frame_5610.jpg
✅ Saved: frames/frame_5640.jpg
✅ Saved: frames/frame_5670.jpg
✅ Saved: frames/frame_5700.jpg
✅ Saved: frames/frame_5730.jpg
✅ Saved: frames/frame_5760.jpg
✅ Saved: frames/frame_5790.jpg
✅ Saved: frames/frame_5820.jpg
✅ Saved: frames/frame_5850.jpg
✅ Saved: frames/frame_5880.jpg
✅ Saved: frames/frame_5910.jpg
✅ Saved: frames/frame_5940.jpg
✅ Saved: frames/frame_5970.jpg
✅ Saved: frames/frame_6000.jpg
✅ Saved: frames/frame_6030.jpg
✅ Saved: fr

Safety Zone Detector

In [ ]:
import numpy as np
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

class SafetyZoneDetector:
    def __init__(self):
        self.model = YOLO("yolov8n-seg.pt")
        self.safety_classes = {
            'sidewalk': {'type': 'safe', 'color': (0, 255, 0), 'alpha': 0.3},
            'road': {'type': 'danger', 'color': (0, 0, 255), 'alpha': 0.3},
            'person': {'type': 'dynamic', 'color': (255, 165, 0), 'alpha': 0.5},
            'car': {'type': 'danger', 'color': (255, 0, 0), 'alpha': 0.5},
            'bicycle': {'type': 'dynamic', 'color': (255, 165, 0), 'alpha': 0.5},
            'traffic light': {'type': 'guide', 'color': (255, 255, 0), 'alpha': 0.5}
        }

    def process_frame(self, frame):
        height, width = frame.shape[:2]
        safety_overlay = np.zeros_like(frame)
        object_overlay = np.zeros_like(frame)
        processed = frame.copy()

        # Run model inference with tracking
        results = self.model.track(frame, persist=True)[0]

        if results.boxes is not None and results.masks is not None:
            try:
                boxes = results.boxes.xyxy.cpu().numpy()
                classes = results.boxes.cls.cpu().numpy()
                masks = results.masks.xy
                track_ids = results.boxes.id.cpu().numpy() if results.boxes.id is not None else None

                for i, (box, cls) in enumerate(zip(boxes, classes)):
                    class_name = self.model.names[int(cls)]
                    safety_info = self.safety_classes.get(
                        class_name,
                        {'type': 'unknown', 'color': (128, 128, 128), 'alpha': 0.5}
                    )

                    if i < len(masks):
                        mask = masks[i]
                        # Ensure mask points are valid
                        if mask is not None and len(mask) > 0:
                            # Convert mask points to integer array
                            mask_points = np.array(mask, dtype=np.int32)

                            # Ensure mask points are properly shaped
                            if mask_points.shape[0] > 0:
                                if class_name == 'sidewalk':
                                    # Fill sidewalk area
                                    cv2.fillPoly(safety_overlay, [mask_points], safety_info['color'])
                                    # Draw outline
                                    cv2.polylines(processed, [mask_points], True, safety_info['color'], 2)
                                    # Add label
                                    x_min, y_min = mask_points.min(axis=0)
                                    cv2.putText(processed, "SAFE ZONE",
                                              (int(x_min), int(y_min) - 10),
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                                              safety_info['color'], 2)
                                else:
                                    # Handle other objects
                                    cv2.fillPoly(object_overlay, [mask_points], safety_info['color'])

                                    # Add tracking ID and label
                                    if track_ids is not None:
                                        track_id = int(track_ids[i])
                                        label = f"ID {track_id} - {class_name}"
                                        x_min, y_min = mask_points.min(axis=0)
                                        cv2.putText(processed, label,
                                                  (int(x_min), int(y_min) - 10),
                                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                                                  safety_info['color'], 2)

                # Blend overlays
                processed = cv2.addWeighted(processed, 1, safety_overlay, 0.3, 0)
                processed = cv2.addWeighted(processed, 1, object_overlay, 0.5, 0)

            except Exception as e:
                print(f"Warning: Error processing frame: {str(e)}")
                return frame  # Return original frame if processing fails

        return processed

# Create output directory
import os
os.makedirs("safety_processed_frames", exist_ok=True)

# Initialize detector
detector = SafetyZoneDetector()

# Process frames
for frame_file in sorted(os.listdir("frames")):
    frame_path = os.path.join("frames", frame_file)
    frame = cv2.imread(frame_path)

    processed_frame = detector.process_frame(frame)

    output_path = os.path.join("safety_processed_frames", frame_file)
    cv2.imwrite(output_path, processed_frame)
    print(f"✅ Processed: {frame_file}")

print("✅ Completed safety processing")


0: 384x640 4 persons, 1 train, 185.0ms
Speed: 5.4ms preprocess, 185.0ms inference, 32.4ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5160.jpg

0: 384x640 2 persons, 1 train, 177.5ms
Speed: 5.0ms preprocess, 177.5ms inference, 20.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5190.jpg

0: 384x640 2 persons, 1 train, 172.5ms
Speed: 4.5ms preprocess, 172.5ms inference, 21.7ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5220.jpg

0: 384x640 2 persons, 1 train, 173.3ms
Speed: 5.6ms preprocess, 173.3ms inference, 17.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5250.jpg

0: 384x640 2 persons, 1 train, 176.8ms
Speed: 4.4ms preprocess, 176.8ms inference, 23.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5280.jpg

0: 384x640 2 persons, 1 train, 188.7ms
Speed: 5.3ms preprocess, 188.7ms inference, 27.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5310.jp

Process frames according to safety analysis


In [ ]:
import os
import cv2

os.makedirs("safety_processed_frames", exist_ok=True)
detector = SafetyZoneDetector()

# Process each frame
frames_dir = "frames"
frame_files = sorted(os.listdir(frames_dir))
for frame_file in frame_files:
    # Read frame
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    # Process with safety detection
    processed_frame = detector.process_frame(frame)

    # Save processed frame
    output_path = os.path.join("safety_processed_frames", frame_file)
    cv2.imwrite(output_path, processed_frame)
    print(f"✅ Processed: {frame_file}")

print("✅ Completed safety processing")



0: 384x640 4 persons, 1 train, 200.3ms
Speed: 3.9ms preprocess, 200.3ms inference, 28.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5160.jpg

0: 384x640 2 persons, 1 train, 178.7ms
Speed: 5.9ms preprocess, 178.7ms inference, 19.2ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5190.jpg

0: 384x640 2 persons, 1 train, 176.0ms
Speed: 6.1ms preprocess, 176.0ms inference, 19.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5220.jpg

0: 384x640 2 persons, 1 train, 181.8ms
Speed: 4.7ms preprocess, 181.8ms inference, 17.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5250.jpg

0: 384x640 2 persons, 1 train, 178.6ms
Speed: 4.6ms preprocess, 178.6ms inference, 24.4ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5280.jpg

0: 384x640 2 persons, 1 train, 182.3ms
Speed: 5.4ms preprocess, 182.3ms inference, 26.8ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed: frame_5310.jp

Creating new video with the implementation

In [ ]:
import cv2
import os

frames_dir = "safety_processed_frames"
output_video = "phenix_walk.mp4"

# Get frame properties
frame_files = sorted(os.listdir(frames_dir))
first_frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
height, width = first_frame.shape[:2]

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 30, (width, height))

# Add frames to video
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    out.write(frame)

out.release()
print(f"✅ Created video: {output_video}")

# Download video (in Colab)
from google.colab import files
files.download(output_video)

✅ Created video: phenix_walk.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Importing from roboflow, establish data.yaml, and train

In [ ]:
# List current directory contents
!ls


# List contents again to see what directory was created
!ls

# Print the directory structure
!tree

# Now let's create the yaml with the correct path
import os

# Create data.yaml with the exact path
data_yaml = """
path: /content/Phoenix Data
train: train/images
val: valid/images
test: test/images

nc: 1
names: ['walkable-zone']
"""

# Save the YAML configuration
with open("Phoenix Data/data.yaml", "w") as f:  # Save it inside Phoenix Data directory
    f.write(data_yaml)

# Verify the paths exist
print("\nVerifying paths...")
base_dir = "/content/Phoenix Data"
train_path = os.path.join(base_dir, "train/images")
valid_path = os.path.join(base_dir, "valid/images")
test_path = os.path.join(base_dir, "test/images")

print(f"Train path exists: {os.path.exists(train_path)}")
print(f"Valid path exists: {os.path.exists(valid_path)}")
print(f"Test path exists: {os.path.exists(test_path)}")

# If paths are verified, train the model
if os.path.exists(train_path) and os.path.exists(valid_path):
    model = YOLO('yolov8n-seg.pt')
    results = model.train(
        data='Phoenix Data/data.yaml',  # Updated path to data.yaml
        epochs=50,
        imgsz=640,
        batch=16,
        name='phoenix_walkable3'
    )
else:
    print("Error: Some required directories are missing!")

 data.yaml  'Phoenix Data'     runs	     videos
 __MACOSX    phoenixdata.zip   sample_data   yolov8n-seg.pt
 data.yaml  'Phoenix Data'     runs	     videos
 __MACOSX    phoenixdata.zip   sample_data   yolov8n-seg.pt
/bin/bash: line 1: tree: command not found

Verifying paths...
Train path exists: True
Valid path exists: True
Test path exists: True
Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=Phoenix Data/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=phoenix_walkable35, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=Fals

train: Scanning /content/Phoenix Data/train/labels... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<00:00, 468.67it/s]

train: New cache created: /content/Phoenix Data/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/Phoenix Data/valid/labels... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<00:00, 1213.63it/s]

...

      50/50         0G     0.5251     0.7489      1.005      1.174          6        640: 100%|██████████| 3/3 [00:45<00:00, 15.31s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]

                   all          3          3      0.983          1      0.995      0.907      0.983          1      0.995      0.895



50 epochs completed in 0.675 hours.
Optimizer stripped from runs/segment/phoenix_walkable35/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/phoenix_walkable35/weights/best.pt, 6.8MB

Validating runs/segment/phoenix_walkable35/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


                   all          3          3      0.981          1      0.995      0.995      0.981          1      0.995      0.929
Speed: 2.7ms preprocess, 319.9ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to runs/segment/phoenix_walkable35


Saving the model based on this fine-tuning

In [ ]:
# Save your trained model weights
model.save('walkable_model.pt')

#download to google colab
from google.colab import files
files.download('walkable_model.pt')

# For next time when you want to continue training:
# model = YOLO('walkable_model.pt')  # Load your saved model instead of yolov8n-seg.pt
# results = model.train(
#   data='data.yaml',
#   epochs=50,
#   imgsz=640,
 #   batch=16,
  # name='phoenix_walkable_continued',
 #   resume=True  # This tells it to continue from your saved weights
# )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Test the model on the video

In [ ]:
# First download the video

!pip uninstall -y torch torchvision ultralytics opencv-python

!pip install ultralytics

!pip install yt-dlp

!pip install opencv-python

import os
import cv2
import numpy as np
from ultralytics import YOLO
import glob

def draw_mask_safely(frame, mask, color, alpha=0.5):
    """Safely draw a mask on a frame with validation"""
    try:
        # Validate mask
        if mask is None or len(mask) < 3:  # Need at least 3 points for a polygon
            return frame

        # Convert to correct format
        mask_points = np.array(mask, dtype=np.int32)
        if mask_points.shape[0] < 3:  # Double-check after conversion
            return frame

        # Create overlay
        overlay = frame.copy()
        cv2.fillPoly(overlay, [mask_points], color)
        return cv2.addWeighted(frame, 1-alpha, overlay, alpha, 0)

    except Exception as e:
        print(f"Warning: Could not draw mask: {e}")
        return frame

os.makedirs("videos", exist_ok=True)
VIDEO_URL = "https://www.youtube.com/watch?v=bZnjrRuBT_k"
!yt-dlp -o "videos/Phoenix_Walk.%(ext)s" {VIDEO_URL}

# Load both models
walkable_model = YOLO('walkable_model.pt')
object_model = YOLO('yolov8n-seg.pt')

# Set up video capture
video_path = glob.glob("videos/Phoenix_Walk.*")[0]
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create output video writer
output_path = 'processed_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Set time range (1:26 to 2:13)
start_sec, end_sec = 86, 133
start_frame = int(start_sec * fps)
end_frame = int(end_sec * fps)

cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
frame_count = 0

while cap.isOpened() and frame_count < (end_frame - start_frame):
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = frame.copy()

    # 1. Detect walkable zones with your fine-tuned model
    walkable_results = walkable_model.predict(frame, show=False)
    if walkable_results[0].masks is not None:
        for mask in walkable_results[0].masks.xy:
            # Safely draw walkable zones
            processed_frame = draw_mask_safely(
                processed_frame,
                mask,
                color=(0, 255, 0),  # Green for walkable zones
                alpha=0.3
            )
            # Add outline if mask is valid
            if len(mask) >= 3:
                cv2.polylines(processed_frame, [np.int32(mask)], True, (0, 255, 0), 2)

    # 2. Detect and track objects with original model
    object_results = object_model.track(frame, persist=True)[0]
    if object_results.boxes is not None and object_results.masks is not None:
        masks = object_results.masks.xy
        track_ids = object_results.boxes.id.cpu().numpy() if object_results.boxes.id is not None else None
        classes = object_results.boxes.cls.cpu().numpy()

        for i, (mask, cls) in enumerate(zip(masks, classes)):
            # Safely draw objects
            processed_frame = draw_mask_safely(
                processed_frame,
                mask,
                color=(255, 165, 0),  # Orange for objects
                alpha=0.5
            )

            # Add tracking ID and label
            if track_ids is not None and len(mask) > 0:
                track_id = int(track_ids[i])
                class_name = object_model.names[int(cls)]
                label = f"ID {track_id} - {class_name}"
                x_min, y_min = mask.min(axis=0)
                cv2.putText(processed_frame, label,
                          (int(x_min), int(y_min) - 10),
                          cv2.FONT_HERSHEY_SIMPLEX,
                          0.6,
                          (255, 255, 255),
                          2)

    out.write(processed_frame)
    frame_count += 1
    if frame_count % 30 == 0:
        print(f"Processed frame {frame_count}")

cap.release()
out.release()

# Download the processed video
from google.colab import files
files.download('processed_video.mp4')

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: ultralytics 8.3.75
Uninstalling ultralytics-8.3.75:
  Successfully uninstalled ultralytics-8.3.75
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
  Using cached ultralytics-8.3.75-py3-none-any.whl.metadata (35 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
Using cached ultralytics-8.3.75-py3-none-any.whl (914 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 11.9 MB/s eta 0:00:00
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
              ^^^^^^^^^

AttributeError: module 'torch' has no attribute '__version__'

Proximity Detector

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

class ProximityDetector:
    def __init__(self):
        # Only need YOLO for object detection
        self.walkable_model = YOLO('walkable_model.pt')  # For walkable areas
        self.object_model = YOLO('yolov8n-seg.pt')
        # Define immediate proximity zone (area right in front)
        self.proximity_height = 0.4  # Bottom 40% of frame
        self.proximity_width = 0.4   # Center 40% of frame width

    def is_in_proximity(self, box, frame_shape):
        """Check if object is very close to robot's path"""
        height, width = frame_shape[:2]

        # Get box coordinates
        x1, y1, x2, y2 = box
        box_center_x = (x1 + x2) / 2
        box_bottom = y2

        # Define immediate front zone
        zone_left = width * (0.5 - self.proximity_width/2)
        zone_right = width * (0.5 + self.proximity_width/2)
        zone_top = height * (1 - self.proximity_height)

        # Check if object is in immediate front zone
        if (zone_left < box_center_x < zone_right and box_bottom > zone_top):
            relative_distance = (box_bottom - zone_top) / (height - zone_top)
            return True, relative_distance
        return False, 0

    def process_frame(self, frame):
        height, width = frame.shape[:2]
        processed = frame.copy()

        # Detect and track objects
        results = self.model.track(frame, persist=True)[0]

        if results.boxes is not None:
            boxes = results.boxes.xyxy.cpu().numpy()
            classes = results.boxes.cls.cpu().numpy()

            # Track closest obstacle
            closest_distance = 0
            has_close_obstacle = False

            for box, cls in zip(boxes, classes):
                class_name = self.model.names[int(cls)]

                # Only consider person, bicycle, car as obstacles
                if class_name in ['person', 'bicycle', 'car']:
                    in_proximity, distance = self.is_in_proximity(box, frame.shape)

                    if in_proximity and distance > 0.3:  # Object significantly in front
                        has_close_obstacle = True
                        if distance > closest_distance:
                            closest_distance = distance
                            # Draw red box around closest obstacle
                            cv2.rectangle(processed,
                                        (int(box[0]), int(box[1])),
                                        (int(box[2]), int(box[3])),
                                        (0, 0, 255), 2)

            # Add warning text if obstacle detected
            if has_close_obstacle:
                warning_text = "Obstacle ahead, please move"
                text_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
                text_x = (width - text_size[0]) // 2
                cv2.putText(processed, warning_text,
                          (text_x, height - 50),
                          cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                          (0, 0, 255), 3)

        return processed

Video

In [ ]:
from ultralytics import YOLO

# Make a video processor
detector = ProximityDetector()

# Open your video
cap = cv2.VideoCapture("processed_video_copy.mp4")  # Use your video filename here
if not cap.isOpened():
    print("Could not open video")
    exit()

# Create video writer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame
    processed_frame = detector.process_frame(frame)
    out.write(processed_frame)

cap.release()
out.release()

# Download the video
from google.colab import files
files.download('output.mp4')

Could not open video


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>